In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/spaceship-titanic/sample_submission.csv
/kaggle/input/spaceship-titanic/train.csv
/kaggle/input/spaceship-titanic/test.csv


In [2]:
#MY WORST MISTAKE HERE WAS TO USE 
#USE TRAIN DATA IN SI INSTEAD
#OF VALID DATA FOR IMPUTATION
#OF VALID X WHICH
#REsulted in increase no of
# rows with nan values

In [3]:
train_data = pd.read_csv("/kaggle/input/spaceship-titanic/train.csv")
test_data = pd.read_csv("/kaggle/input/spaceship-titanic/test.csv")
train_data.Transported = train_data.Transported.astype(int)
X = train_data.copy()
y = train_data.copy()
X = X.drop("Transported",axis=1)
y = train_data.Transported


In [4]:
#import library 
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error

In [5]:
#train test split
train_X,valid_X,train_y,valid_y = train_test_split(X,y,random_state = 1)


In [6]:
#divide numerical and categorical data
numerical_data = [c for c in train_X.columns
                 if train_X[c].dtypes in["int","float"]]
print(numerical_data)
categorical_data = [c for c in train_X.columns
                   if train_X[c].dtypes in ["object"]
                   and train_X[c].nunique() < 10
                   and set(valid_X[c]).issubset(set(train_X[c]))]
                   


['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']


In [7]:
#fill missing values with imputation
si = SimpleImputer()
i_n_train_X = pd.DataFrame(si.fit_transform(train_X[numerical_data]))
i_n_valid_X = pd.DataFrame(si.transform(valid_X[numerical_data]))
i_n_test = pd.DataFrame(si.transform(test_data[numerical_data]))
i_n_train_X.columns = train_X[numerical_data].columns
i_n_valid_X.columns= valid_X[numerical_data].columns
i_n_test.columns = test_data[numerical_data].columns
#fill missing categorical values
si2 = SimpleImputer(strategy="most_frequent")
i_c_train_X = pd.DataFrame(si2.fit_transform(train_X[categorical_data]))
i_c_valid_X = pd.DataFrame(si2.transform(valid_X[categorical_data]))
i_c_test = pd.DataFrame(si2.transform(test_data[categorical_data]))
i_c_train_X.columns = train_X[categorical_data].columns
i_c_valid_X.columns = valid_X[categorical_data].columns
i_c_test.columns = test_data[categorical_data].columns

In [8]:
i_c_test

,HomePlanet,CryoSleep,Destination,VIP
0,Earth,True,TRAPPIST-1e,False
1,Earth,False,TRAPPIST-1e,False
2,Europa,True,55 Cancri e,False
3,Europa,False,TRAPPIST-1e,False
4,Earth,False,TRAPPIST-1e,False
...,...,...,...,...
4272,Earth,True,TRAPPIST-1e,False
4273,Earth,False,TRAPPIST-1e,False
4274,Mars,True,55 Cancri e,False
4275,Europa,False,TRAPPIST-1e,False


In [9]:
#one-hot encode the categorical data
oh = OneHotEncoder(handle_unknown='ignore', sparse=False)
oh_train_X = pd.DataFrame(oh.fit_transform(i_c_train_X))
oh_valid_X = pd.DataFrame(oh.transform(i_c_valid_X))
oh_test = pd.DataFrame(oh.transform(i_c_test))
#put back index
oh_train_X.index = i_c_train_X.index
oh_valid_X.index = i_c_valid_X.index
oh_test.index = i_c_test.index
#remove categorical data from test data
#combine categorical and numerical data of train and validation 
final_train_X = pd.concat([i_n_train_X , oh_train_X],axis=1)
final_valid_X = pd.concat([i_n_valid_X,oh_valid_X],axis=1)
final_test = pd.concat([i_n_test,oh_test],axis=1)
i_n_test

/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
0,27.000000,0.0,0.0,0.0,0.0,0.0
1,19.000000,0.0,9.0,0.0,2823.0,0.0
2,31.000000,0.0,0.0,0.0,0.0,0.0
3,38.000000,0.0,6652.0,0.0,181.0,585.0
4,20.000000,10.0,0.0,635.0,0.0,0.0
...,...,...,...,...,...,...
4272,34.000000,0.0,0.0,0.0,0.0,0.0
4273,42.000000,0.0,847.0,17.0,10.0,144.0
4274,28.884953,0.0,0.0,0.0,0.0,0.0
4275,28.884953,0.0,2680.0,0.0,0.0,523.0


In [10]:
final_test

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,0,1,2,3,4,5,6,7,8,9
0,27.000000,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
1,19.000000,0.0,9.0,0.0,2823.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
2,31.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0
3,38.000000,0.0,6652.0,0.0,181.0,585.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
4,20.000000,10.0,0.0,635.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,34.000000,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
4273,42.000000,0.0,847.0,17.0,10.0,144.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
4274,28.884953,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0
4275,28.884953,0.0,2680.0,0.0,0.0,523.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0


In [11]:
final_train_X

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,0,1,2,3,4,5,6,7,8,9
0,28.0,0.000000,0.0,0.000000,0.0,0.000000,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0
1,12.0,0.000000,0.0,0.000000,0.0,308.308127,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
2,27.0,994.000000,0.0,26.000000,1.0,0.000000,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
3,2.0,0.000000,0.0,0.000000,0.0,0.000000,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
4,40.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6514,38.0,886.000000,0.0,618.000000,0.0,0.000000,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
6515,39.0,0.000000,261.0,170.119792,1831.0,1797.000000,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
6516,24.0,229.216969,32.0,8.000000,0.0,0.000000,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
6517,47.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0


In [12]:
final_test

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,0,1,2,3,4,5,6,7,8,9
0,27.000000,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
1,19.000000,0.0,9.0,0.0,2823.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
2,31.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0
3,38.000000,0.0,6652.0,0.0,181.0,585.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
4,20.000000,10.0,0.0,635.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,34.000000,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
4273,42.000000,0.0,847.0,17.0,10.0,144.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
4274,28.884953,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0
4275,28.884953,0.0,2680.0,0.0,0.0,523.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0


In [13]:
#data preprocessing completed 

In [14]:
#train model for val
my_model = XGBRegressor(n_estimators=500, learning_rate=0.05)
my_model.fit(final_train_X, train_y,early_stopping_rounds=10, 
             eval_set=[(final_valid_X, valid_y)])

[0]	validation_0-rmse:0.49014
[1]	validation_0-rmse:0.48117
[2]	validation_0-rmse:0.47276
[3]	validation_0-rmse:0.46478
[4]	validation_0-rmse:0.45772
[5]	validation_0-rmse:0.45093
[6]	validation_0-rmse:0.44461
[7]	validation_0-rmse:0.43899
[8]	validation_0-rmse:0.43341
[9]	validation_0-rmse:0.42868
[10]	validation_0-rmse:0.42408
[11]	validation_0-rmse:0.41983
[12]	validation_0-rmse:0.41604
[13]	validation_0-rmse:0.41236
[14]	validation_0-rmse:0.40914
[15]	validation_0-rmse:0.40616
[16]	validation_0-rmse:0.40336
[17]	validation_0-rmse:0.40106


/opt/conda/lib/python3.10/site-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[18]	validation_0-rmse:0.39872
[19]	validation_0-rmse:0.39652
[20]	validation_0-rmse:0.39466
[21]	validation_0-rmse:0.39305
[22]	validation_0-rmse:0.39116
[23]	validation_0-rmse:0.38977
[24]	validation_0-rmse:0.38825
[25]	validation_0-rmse:0.38712
[26]	validation_0-rmse:0.38584
[27]	validation_0-rmse:0.38489
[28]	validation_0-rmse:0.38369
[29]	validation_0-rmse:0.38297
[30]	validation_0-rmse:0.38201
[31]	validation_0-rmse:0.38128
[32]	validation_0-rmse:0.38075
[33]	validation_0-rmse:0.38013
[34]	validation_0-rmse:0.37961
[35]	validation_0-rmse:0.37900
[36]	validation_0-rmse:0.37858
[37]	validation_0-rmse:0.37814
[38]	validation_0-rmse:0.37770
[39]	validation_0-rmse:0.37740
[40]	validation_0-rmse:0.37696
[41]	validation_0-rmse:0.37669
[42]	validation_0-rmse:0.37634
[43]	validation_0-rmse:0.37623
[44]	validation_0-rmse:0.37597
[45]	validation_0-rmse:0.37591
[46]	validation_0-rmse:0.37565
[47]	validation_0-rmse:0.37543
[48]	validation_0-rmse:0.37521
[49]	validation_0-rmse:0.37507
[50]	val

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.05, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=500, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [15]:
best_n = my_model.best_iteration
best_n

72

In [16]:
#combine validation and training 
#data
final_X = pd.concat([final_train_X,final_valid_X],axis=0)
final_y = pd.concat([train_y,valid_y],axis=0)


In [17]:
#final model

In [18]:
model = XGBRegressor(n_estimators=500, learning_rate=0.05)
model.fit(final_X,final_y)
pred = model.predict(final_test)
pred

array([ 0.6421576 , -0.01145012,  0.9697864 , ...,  0.9528676 ,
        0.5669556 ,  0.31897107], dtype=float32)

In [19]:
from sklearn.ensemble import RandomForestRegressor
final_X.columns = final_X.columns.astype(str)
final_test.columns= final_test.columns.astype(str)
lmodel= RandomForestRegressor(n_estimators=500)
lmodel.fit(final_X,final_y)
prel= lmodel.predict(final_test)
prel

array([0.6466148 , 0.        , 1.        , ..., 0.98559067, 0.908     ,
       0.10216667])

In [20]:
#convert the prediction to
#bool
threshold = 0.5
#threshold to convert probabilities to binary predictions
binary_predictions = (pred >= threshold).astype(bool)
binary_predictions

array([ True, False,  True, ...,  True,  True, False])

In [21]:
test_df = pd.read_csv("/kaggle/input/spaceship-titanic/test.csv")
output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Transported': binary_predictions})
output.to_csv('/kaggle/working/submission.csv', index=False)